In [6]:
import pandas as pd
import seaborn as sns


## 그룹화 및 Aggregation

### 1. 그룹화
- `dataframe.groupby()`: SQL의 GROUP BY절과 유사. 그룹화 후 합계(sum), 평균(mean), 개수(count) 등 다양한 집계(aggregation) 함수를 적용할 수 있음
- 단일 그룹화
- 다중 그룹화  
    (예시1)

### 2. Aggregation
- 각 그룹에 대해 여러 가지 통계를 한 번에 보고 싶다면 `agg()` 메서드를 사용

In [ ]:
tips = sns.load_dataset('tips')
# 단일 조건
tips.groupby('day')['tip'].mean() # 요일을 기준으로 팁 평균
tips.groupby('smoker')['total_bill'].mean() # 흡연 여부 별로 total_bill의 평균
tips.groupby('time')['day'].sum() # time 별 day의 합계 # 문자열은 집계불가능하므로 오류 (정수형만 가능)

# 다중 조건
tips.groupby('day')['tip'].agg(['mean', 'sum']) # day를 인덱스로 출력
result = tips.groupby(['day', 'smoker'])['tip'].count() # 여러 개의 기준을 가지고 정렬, 인덱스가 여러개, 멀티 인덱스
result.reset_index(name = 'count') # 인덱스를 컬럼으로 가져 옴 # name = 을 이용하여 이름이 없는 컬럼명을 지정 가능, 아무것도 없을 때는 위에서 집계했던 컬럼명으로 출력

# Aggregation
tips.groupby('day').agg({ 'total_bill': 'sum', 'tip': 'mean'})

## 문자열 처리

### 1. `Series.str` 접근자를 통해 이용할 수 있음
- 1개의 문자열에 많은 데이터가 들어가 있는데 데이터 하나하나 전부 처리하기 위해 str을 삽입  
    (예시2)


In [ ]:
#예시2
products = pd.Series([
    "Bush Somerset Collection Bookcase",
    "Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",
    "Self-Adhesive Address Labels",
    "Staples 8.5x11 Copy Paper"
])

result = products.str.contains('Bookcase')
result.tolist() # 리스트 형태로 변형
products.str.upper()
products.str.lower()
products.str.title()
products.str.strip() # strip('x') 특정문자 제거
products.str.replace('a', 'b') # regex = False
products.str.split(' ') # str[0] 으로 첫부분 추출 가능
products.str.title()
products.str.cat(sep=',')
products.str.len()


## 시간 데이터 처리

### 1. 날짜 형식 변환
- Pandas는 NumPy의 datetime64 타입을 기반으로, Timestamp(타임스탬프)와 DatetimeIndex(시간 인덱스) 등을 사용해 시계열 데이터를 효율적으로 처리
- `pd.to_datetime()`: 날짜 형식 변환 
- `to_datetime`은 형식을 자동 인식하려 함 but  실패할 경우 format 인자로 형식을 지정해 줄 수도 있습니다 (예: format='%d/%m/%Y').
    (예시3)

### 2. DatetimeIndex와 시계열
- 날짜 데이터를 인덱스로 설정하면 시계열 사용 가능
- DatetimeIndex 상태에서는 .loc['2016-11'] 이런 식으로 연-월 지정으로 데이터 추출 가능
- 접근자
    - `Series.dt.year`, `month`, `day`, `hour`, `minute` 등: 연, 월, 일, 시, 분...
    - `Series.dt.day_name()`: 요일 이름 (예: Monday)
    - `Series.dt.weekday`: 요일 번호 (월=0,...일=6)
    - `Series.dt.quarter`: 분기(1~4)
    - `Series.dt.days_in_month`: 그 달의 일 수 등.  
    (예시4)

In [ ]:
#예시3
orders = pd.read_csv('superstore_orders.csv')
orders['Order Date'] = pd.to_datetime(orders['Order Date'], format = '%d/%m/%Y') # pandas 데이트 타임 포멧은 무조건 일,월,년 순


In [ ]:
#예시4
orders = orders.set_index('Order Date') # 인덱스 지정
orders = orders.reset_index('Order Date') # 컬럼으로 재지정

## 데이터 결합

### 1. merge / join
- `pd.merge(df1, df2, how='inner', on='키')`: 'inner', 'left', 'right', 'outer' 등을 지정 가능 
- `df1.merge(df2, ...)`:  `left_on`과 `right_on`을 각각 지정 가능. 혹은 인덱스를 키로 쓰고 싶으면 `left_index=True`, `right_index=True` 옵션을 사용
- `join()`: 인덱스 기준으로 정렬

### 2. Concatenate
- `pd.concat([df1, df2, ...], axis=0)`: 행 방향으로 이어붙임. 동일한 둘 이상의 데이터프레임을 위아래로 쌓는다
- `ignore_index=True`: 인덱스가 겹치는 것을 무시
- `axis=1`: 열 방향으로 옆으로 붙임. 맞지 않는 인덱스는 NaN  
        


#### 접미사 지정 
- merge 결과 중 _x, _y로 접미사가 붙는 컬럼  
    suffixes=('_order','_return') 같이 직접 접미사를 지정 가능  
        (예시5)

In [ ]:
#예시5
pd.merge(df1, df2, how='left', on='키') # = df1.merge(df2, how='left', on='키')


# 주문 테이블
orders = pd.DataFrame({
    'Order ID': [101, 102, 103],
    'Product': ['Bookcase', 'Chair', 'Lamp'],
    'Profit': [100, 50, 20]
})
# 반품된 주문 테이블
returns = pd.DataFrame({
    'Order ID': [102, 105],
    'Returned': ['Yes', 'Yes']
})

merged = orders.merge(returns, on = 'Order ID', how = 'left')
merged['Returned'] = merged['Returned'].fillna('No') # 결축처리
df1 = pd.DataFrame({'City': ['Seoul','Busan'], 'Sales':[100,150]})
df2 = pd.DataFrame({'City': ['Daegu','Incheon'], 'Sales':[80,70]})
result = pd.concat([df1,df2], ignore_index = True)



orders = pd.DataFrame({
    'Order ID': [101, 102, 103],
    'Product': ['Bookcase', 'Chair', 'Lamp'],
    'Profit': [100, 50, 20]
})
returns = pd.DataFrame({
    'Order ID': [102, 105],
    'Returned': ['Yes', 'Yes'],
    'Profit': [4, 44]
})

merged = orders.merge(returns, on='Order ID', how='left')
merged = orders.merge(returns, on='Order ID', how='left', suffixes=('_order','_return'))


## 05. apply, map, lambda 활용

### 1. Apply 함수
- DataFrame.apply(func, axis=0): 각 열(column)에 대해 함수를 적용 (default). axis=1로 지정하면 각 행(row)에 대해 함수를 적용.
- Series.apply(func): Series의 각 값에 함수를 적용한 결과 Series를 반환  
        (예시6)

### 2. map 함수
- Series.map(func or dict):  Series의 각 요소를 주어진 함수에 매핑하여 새로운 Series를 반환. 단, map은 원소별 동작만 가능하고 index나 다른 컬럼 정보는 접근 불가 (그래서 행 종합 처리는 apply(axis=1) 써야 함).  
        (예시7)

### 3. 

In [ ]:
#예시6
tips['tip_ratio'] = tips['tip'] / tips['total_bill']
tips['new_ratio'] = tips.apply(lambda row : row['tip'] / row['total_bill'], axis = 1 )

In [ ]:
#예시7
import seaborn as sns
tips = sns.load_dataset('tips')

tips['sex'].map({'Male':1, 'Female':0}) # 여자 0 / 남자 1
tips['smoker'].map({'No': 0, 'Yes': 1}) # 비흡연자 0 / 흡연자 1
tips['price_label'] = tips['total_bill'].map(lambda x: 'expensive' if x >= 20 else 'cheap')
tips[['total_bill','price_label']].head(5)